In [36]:
import os
import re
import zipfile
import json
import torch
from unidecode import unidecode
import spacy
from util.file_util import cached_path, get_root_path
from dataloader import Dataloader
from jointBERT import JointBERT
from postprocess import recover_intent, recover_slot
from spacy.symbols import ORTH, LEMMA, POS

In [37]:
class BERTNLU():
    def __init__(self, mode,folder_name,config_file,model_file):
        assert mode == 'usr' or mode == 'sys' or mode == 'all'
        self.mode = mode
        current_folder = globals()['_dh'][0]+"/"+folder_name
        config_file = os.path.join(current_folder, 'configs/{}'.format(config_file))
        config = json.load(open(config_file))
        # print(config['DEVICE'])
        # DEVICE = config['DEVICE']
        DEVICE = 'cpu' if not torch.cuda.is_available() else 'cuda:0'
        root_dir = os.path.dirname(current_folder)
        data_dir = os.path.join(root_dir, config['data_dir'])
        output_dir = os.path.join(root_dir, config['output_dir'])

        intent_vocab = json.load(open(os.path.join(data_dir, 'intent_vocab.json')))
        tag_vocab = json.load(open(os.path.join(data_dir, 'tag_vocab.json')))
        dataloader = Dataloader(intent_vocab=intent_vocab, tag_vocab=tag_vocab,
                                pretrained_weights=config['model']['pretrained_weights'])

        best_model_path = os.path.join(output_dir, 'pytorch_model.bin')
        if not os.path.exists(best_model_path):
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            print('Load from model_file param')
            archive_file = cached_path(model_file)
            archive = zipfile.ZipFile(archive_file, 'r')
            archive.extractall(root_dir)
            archive.close()
        # print('Load from', best_model_path)
        model = JointBERT(config['model'], DEVICE, dataloader.tag_dim, dataloader.intent_dim)
        model.load_state_dict(torch.load(os.path.join(output_dir, 'pytorch_model.bin'), DEVICE))
        model.to(DEVICE)
        model.eval()

        self.model = model
        self.use_context = config['model']['context']
        self.dataloader = dataloader
        self.nlp = spacy.load('en_core_web_sm')
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except Exception:
            print('download en_core_web_sm for spacy')
            from spacy.cli.download import download as spacy_download
            spacy_download("en_core_web_sm")
            spacy_model_module = __import__("en_core_web_sm")
            self.nlp = spacy_model_module.load()
        with open(os.path.join(globals()['_dh'][0], 'db/postcode.json'), 'r') as f:
            token_list = json.load(f)

        for token in token_list:
            token = token.strip()
            self.nlp.tokenizer.add_special_case(token, [{ORTH: token, LEMMA: token, POS: u'NOUN'}])
        print("BERTNLU loaded")

    def predict(self, utterance, context=list()):
        # Note: spacy cannot tokenize 'id' or 'Id' correctly.
        utterance = re.sub(r'\b(id|Id)\b', 'ID', utterance)
        # tokenization first, very important!
        ori_word_seq = [token.text for token in self.nlp(unidecode(utterance)) if token.text.strip()]
        # print(ori_word_seq)
        ori_tag_seq = ['O'] * len(ori_word_seq)
        if self.use_context:
            if len(context) > 0 and type(context[0]) is list and len(context[0]) > 1:
                context = [item[1] for item in context]
            context_seq = self.dataloader.tokenizer.encode('[CLS] ' + ' [SEP] '.join(context[-3:]))
            context_seq = context_seq[:512]
        else:
            context_seq = self.dataloader.tokenizer.encode('[CLS]')
        intents = []
        da = {}

        word_seq, tag_seq, new2ori = self.dataloader.bert_tokenize(ori_word_seq, ori_tag_seq)
        word_seq = word_seq[:510]
        tag_seq = tag_seq[:510]
        batch_data = [[ori_word_seq, ori_tag_seq, intents, da, context_seq,
                       new2ori, word_seq, self.dataloader.seq_tag2id(tag_seq), self.dataloader.seq_intent2id(intents)]]

        pad_batch = self.dataloader.pad_batch(batch_data)
        pad_batch = tuple(t.to(self.model.device) for t in pad_batch)
        word_seq_tensor, tag_seq_tensor, intent_tensor, word_mask_tensor, tag_mask_tensor, context_seq_tensor, context_mask_tensor = pad_batch
        slot_logits, intent_logits = self.model.forward(word_seq_tensor, word_mask_tensor,
                                                        context_seq_tensor=context_seq_tensor,
                                                        context_mask_tensor=context_mask_tensor)
        das = recover_intent(self.dataloader, intent_logits[0])
        das2 = recover_slot(self.dataloader, intent_logits[0], slot_logits[0], tag_mask_tensor[0],
                     batch_data[0][0], batch_data[0][-4])
        dialog_act = []
        for intent, slot, value in das2:
            domain = intent.split('-')[0]
            result={}
            result["intent"]=intent
            # result["domain"]=domain
            result["slot"]=slot
            result["value"]=value
            dialog_act.append(result)
        return dialog_act

In [38]:
text_list=[
    "How about rosa's bed and breakfast ? Their postcode is cb22ha.",
    "I want a bus to get to Mira Mesa.",
    "I would like a taxi from Gilman drive to Los angeles."
]
nlu = BERTNLU(mode='all',folder_name="multiwoz22",config_file='multiwoz_all_context.json',model_file='output/all_context/bert_multiwoz_all_context.zip')

bert-base-uncased
BERTNLU loaded


In [39]:
for text in text_list:
    predictions=nlu.predict(text)
    print("Text:",text)
    for i in range(len(predictions)):
        print("Pred ",i,":",json.dumps(predictions[i],indent=4))

Text: How about rosa's bed and breakfast ? Their postcode is cb22ha.
Pred  0 : {
    "intent": "Hotel-Recommend",
    "slot": "Name",
    "value": "rosa"
}
Pred  1 : {
    "intent": "Hotel-Inform",
    "slot": "Post",
    "value": "cb22ha"
}
Text: I want a bus to get to Mira Mesa.
Pred  0 : {
    "intent": "Taxi-Inform",
    "slot": "Dest",
    "value": "Mira Mesa"
}
Text: I would like a taxi from Gilman drive to Los angeles.
Pred  0 : {
    "intent": "Taxi-Inform",
    "slot": "Depart",
    "value": "Gilman drive"
}


In [40]:
for j in range(len(text_list)):
    text=text_list[j]
    predictions=nlu.predict(text,context=text_list[:j])
    print("Text:",text)
    print("Context:",text_list[:j])
    for i in range(len(predictions)):
        print("Pred ",i,":",json.dumps(predictions[i],indent=4))

Text: How about rosa's bed and breakfast ? Their postcode is cb22ha.
Context: []
Pred  0 : {
    "intent": "Hotel-Recommend",
    "slot": "Name",
    "value": "rosa"
}
Pred  1 : {
    "intent": "Hotel-Inform",
    "slot": "Post",
    "value": "cb22ha"
}
Text: I want a bus to get to Mira Mesa.
Context: ["How about rosa's bed and breakfast ? Their postcode is cb22ha."]
Pred  0 : {
    "intent": "Taxi-Inform",
    "slot": "Dest",
    "value": "Mira"
}
Text: I would like a taxi from Gilman drive to Los angeles.
Context: ["How about rosa's bed and breakfast ? Their postcode is cb22ha.", 'I want a bus to get to Mira Mesa.']
Pred  0 : {
    "intent": "Taxi-Inform",
    "slot": "Depart",
    "value": "Gilman drive"
}
